In [1]:
using Pkg
# Activate environment that has ProgressiveHedging installed
Pkg.activate("..")
Pkg.status()

 Activating environment at `~/jdev/ProgressiveHedging/Project.toml`


Project ProgressiveHedging v0.4.0
Status `~/jdev/ProgressiveHedging/Project.toml`
  [a93c6f00] DataFrames v0.20.2
  [b6b21f68] Ipopt v0.6.1
  [4076af6c] JuMP v0.21.2
  [b8f27783] MathOptInterface v0.9.13
  [a759f4b9] TimerOutputs v0.5.3
  [8ba89e20] Distributed 
  [de0858da] Printf 


In [2]:
using Distributed
const WORKERS = 1 # Change to > 1 to use parallel
if nworkers() < WORKERS
    diff = (nprocs() == nworkers() ? WORKERS : WORKERS - nworkers())
    println("Adding $diff worker processes.")
    Distributed.addprocs(diff)
    # Make sure these workers also have an environment with PH installed
    @everywhere using Pkg
    for w in workers()
        @spawnat(w, Pkg.activate(".."))
    end
end

@everywhere using ProgressiveHedging
const PH = ProgressiveHedging

@everywhere using Ipopt
@everywhere using JuMP

### Model Construction Function Definition

In [3]:
function variable_dict()
    first_stage_vars = ["x[1]", "x[2]", "x[3]"]
    second_stage_vars = ["y"]
    third_stage_vars = ["z[1]", "z[2]"]

    var_dict = Dict(1=>first_stage_vars,
                    2=>second_stage_vars,
                    3=>third_stage_vars)
    return var_dict
end

@everywhere function create_model(scenario_id::PH.ScenarioID,
        additional_arg::String; key_word_arg::String="Default key word argument", kwargs...
        )
    
    model = JuMP.Model(()->Ipopt.Optimizer(print_level=0, tol=1e-12))
    
    println(additional_arg)
    println(key_word_arg)
    
    c = [1.0, 10.0, 0.01]
    d = 7.0
    a = 16.0

    α = 1.0
    β = 1.0
    γ = 1.0
    δ = 1.0
    ϵ = 1.0

    s1 = 8.0
    s2 = 4.0
    s11 = 9.0
    s12 = 16.0
    s21 = 5.0
    s22 = 18.0
    
    stage1 = JuMP.@variable(model, x[1:3] >= 0.0)
    JuMP.@constraint(model, x[3] <= 1.0)
    obj = zero(JuMP.GenericQuadExpr{Float64,JuMP.VariableRef})
    JuMP.add_to_expression!(obj, sum(c.*x))

    # Second stage
    stage2 = Vector{JuMP.VariableRef}()
    if scenario_id < PH.scid(2)
        vref = JuMP.@variable(model, y >= 0.0)
        JuMP.@constraint(model, α*sum(x) + β*y >= s1)
        JuMP.add_to_expression!(obj, d*y)
    else
        vref = JuMP.@variable(model, y >= 0.0)
        JuMP.@constraint(model, α*sum(x) + β*y >= s2)
        JuMP.add_to_expression!(obj, d*y)
    end
    push!(stage2, vref)

    # Third stage
    stage3 = Vector{JuMP.VariableRef}()
    if scenario_id == PH.scid(0)
        vref = JuMP.@variable(model, z[1:2])
        JuMP.@constraint(model, ϵ*sum(x) + γ*y + δ*sum(z) == s11)
        JuMP.add_to_expression!(obj, a*sum(z[i]^2 for i in 1:2))
        
    elseif scenario_id == PH.scid(1)
        vref = JuMP.@variable(model, z[1:2])
        JuMP.@constraint(model, ϵ*sum(x) + γ*y + δ*sum(z) == s12)
        JuMP.add_to_expression!(obj, a*sum(z[i]^2 for i in 1:2))

    elseif scenario_id == PH.scid(2)
        vref = JuMP.@variable(model, z[1:2])
        JuMP.@constraint(model, ϵ*sum(x) + γ*y + δ*sum(z) == s21)
        JuMP.add_to_expression!(obj, a*sum(z[i]^2 for i in 1:2))

    else
        vref = JuMP.@variable(model, z[1:2])
        JuMP.@constraint(model, ϵ*sum(x) + γ*y + δ*sum(z) == s22)
        JuMP.add_to_expression!(obj, a*sum(z[i]^2 for i in 1:2))
    end
    append!(stage3, vref)

    JuMP.@objective(model, Min, obj)
    
    vdict = Dict{PH.StageID, Vector{JuMP.VariableRef}}([PH.stid(1) => stage1,
                                                        PH.stid(2) => stage2,
                                                        PH.stid(3) => stage3,
                                                        ])
    
    return JuMPSubproblem(model, scenario_id, vdict)
end
;

### Build Scenario Tree
Note that you must call add_leaf when adding a leaf node instead of add_node.  Failing to do so will cause undefined behavior.

In [4]:
function build_scen_tree()

    probs = [0.5*0.75, 0.5*0.25, 0.5*0.75, 0.5*0.25]
    
    tree = PH.ScenarioTree()
    
    for k in 1:2
        node2 = PH.add_node(tree, tree.root)
        for l in 1:2
            PH.add_leaf(tree, node2, probs[(k-1)*2 + l])
        end
    end
    return tree
end
;

### Extensive Form Solve

In [5]:
ef_model = PH.solve_extensive(build_scen_tree(),
    create_model, 
    ()->Ipopt.Optimizer(print_level=0),
    "Unused example string",
)
println(ef_model)

Unused example string
Default key word argument
Unused example string
Default key word argument
Unused example string
Default key word argument
Unused example string
Default key word argument

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

Min 6 z[1]_{2}² + 6 z[2]_{2}² + 2 z[1]_{1}² + 2 z[2]_{1}² + 6 z[1]_{0}² + 6 z[2]_{0}² + 2 z[1]_{3}² + 2 z[2]_{3}² + x[1]_{0,1,2,3} + 10 x[2]_{0,1,2,3} + 0.01 x[3]_{0,1,2,3} + 3.5 y_{2,3} + 3.5 y_{0,1}
Subject to
 x[1]_{0,1,2,3} + x[2]_{0,1,2,3} + x[3]_{0,1,2,3} + y_{2,3} + z[1]_{2} + z[2]_{2} = 5.0
 x[1]_{0,1,2,3} + x[2]_{0,1,2,3} + x[3]_{0,1,2,3} + y_{0,1} + z[1]_{1} + z[2]_{1} = 16.0
 x[1]_{0,1,2,3} + x[2]_{0,1,2,3

In [6]:
println(JuMP.termination_status(ef_model))
println(JuMP.objective_value(ef_model))
for var in JuMP.all_variables(ef_model)
    println("$var = $(JuMP.value(var))")
end

LOCALLY_SOLVED
178.35374984762046
y_{2,3} = 0.0
x[1]_{0,1,2,3} = 7.562500011769157
x[2]_{0,1,2,3} = 0.0
x[3]_{0,1,2,3} = 1.000000007465833
z[1]_{2} = -1.7812499999655194
z[2]_{2} = -1.7812499999655194
y_{0,1} = 1.7499999908010666
z[1]_{1} = 2.843749999842752
z[2]_{1} = 2.843749999842752
z[1]_{0} = -0.656250000157248
z[2]_{0} = -0.656250000157248
z[1]_{3} = 4.718750000034481
z[2]_{3} = 4.718750000034481


### PH Solve

In [7]:
(n, err, obj, soln, phd) = PH.solve(build_scen_tree(),
                                    create_model,
                                    25.0, "Passed to constructor.",
                                    atol=1e-8, rtol=1e-12, max_iter=500, report=1,
                                    key_word_arg="We changed this key word arg!")
println("Number of iterations: ", n)
println("L^2 error: ", err)
println(obj)

Initializing...
...building submodels...
Passed to constructor.
We changed this key word arg!
Passed to constructor.
We changed this key word arg!
Passed to constructor.
We changed this key word arg!
Passed to constructor.
We changed this key word arg!
...computing starting values...
...augmenting objectives...
Solving...
Iter:    0   AbsR: 4.340309e+00   RelR: 5.144070e-01   Xhat: 3.799774e+00   X: 2.097618e+00
Iter:    1   AbsR: 1.481594e+00   RelR: 1.849188e-01   Xhat: 6.743548e-01   X: 1.319229e+00
Iter:    2   AbsR: 5.299999e-01   RelR: 6.701363e-02   Xhat: 2.692031e-01   X: 4.565410e-01
Iter:    3   AbsR: 3.623073e-01   RelR: 4.551418e-02   Xhat: 2.150947e-01   X: 2.915491e-01
Iter:    4   AbsR: 2.771676e-01   RelR: 3.442779e-02   Xhat: 1.673351e-01   X: 2.209544e-01
Iter:    5   AbsR: 2.179884e-01   RelR: 2.681599e-02   Xhat: 1.099957e-01   X: 1.882017e-01
Iter:    6   AbsR: 1.659724e-01   RelR: 2.027443e-02   Xhat: 5.723030e-02   X: 1.557933e-01
Iter:    7   AbsR: 1.398478e-01 

Iter:   86   AbsR: 5.233350e-07   RelR: 6.920130e-08   Xhat: 4.529441e-07   X: 2.621472e-07
Iter:   87   AbsR: 4.165965e-07   RelR: 5.508713e-08   Xhat: 3.601070e-07   X: 2.094652e-07
Iter:   88   AbsR: 3.321094e-07   RelR: 4.391529e-08   Xhat: 2.864456e-07   X: 1.680642e-07
Iter:   89   AbsR: 2.652737e-07   RelR: 3.507751e-08   Xhat: 2.280039e-07   X: 1.355890e-07
Iter:   90   AbsR: 2.082633e-07   RelR: 2.753895e-08   Xhat: 1.806488e-07   X: 1.036322e-07
Iter:   91   AbsR: 1.646561e-07   RelR: 2.177271e-08   Xhat: 1.425839e-07   X: 8.234968e-08
Iter:   92   AbsR: 1.316630e-07   RelR: 1.740998e-08   Xhat: 1.135323e-07   X: 6.667507e-08
Iter:   93   AbsR: 1.056604e-07   RelR: 1.397163e-08   Xhat: 9.059331e-08   X: 5.437812e-08
Iter:   94   AbsR: 8.499332e-08   RelR: 1.123879e-08   Xhat: 7.232463e-08   X: 4.464316e-08
Iter:   95   AbsR: 6.854051e-08   RelR: 9.063207e-09   Xhat: 5.774873e-08   X: 3.691727e-08
Iter:   96   AbsR: 5.542966e-08   RelR: 7.329541e-09   Xhat: 4.610951e-08   X: 3

In [8]:
aobj = PH.retrieve_aug_obj_value(phd)
println("Augmented Objective: ", aobj)
println("Difference: ", aobj - obj)

Augmented Objective: 178.35374984010076
Difference: 1.398316072709349e-6


In [9]:
soln

,variable,value,stage,scenarios
,String,Float64,Int64,String
1,x[3],1.0,1,"0,1,2,3"
2,x[2],4.28483e-9,1,"0,1,2,3"
3,x[1],7.5625,1,"0,1,2,3"
4,y,1.75,2,"0,1"
5,z[2],-0.65625,3,0
6,z[1],-0.65625,3,0
7,z[1],2.84375,3,1
8,z[2],2.84375,3,1
9,y,9.87429e-9,2,"2,3"


In [10]:
PH.retrieve_no_hats(phd)

,variable,value,stage,scenario,index
,String,Float64,Int64,Int64,Int64
1,x[1],7.5625,1,0,1
2,x[2],0.0,1,0,2
3,x[3],1.0,1,0,3
4,x[1],7.5625,1,1,1
5,x[2],0.0,1,1,2
6,x[3],1.0,1,1,3
7,x[1],7.5625,1,2,1
8,x[2],0.0,1,2,2
9,x[3],1.0,1,2,3


In [11]:
PH.retrieve_w(phd)

,variable,value,stage,scenario,index
,String,Float64,Int64,Int64,Int64
1,W_x[1],-22.0,1,0,1
2,W_x[2],-17.4055,1,0,2
3,W_x[3],-21.5047,1,0,3
4,W_x[1],90.0,1,1,1
5,W_x[2],81.0849,1,1,2
6,W_x[3],90.4949,1,1,3
7,W_x[1],-58.0,1,2,1
8,W_x[2],-56.6228,1,2,2
9,W_x[3],-58.4952,1,2,3
